# Artwork sequence prediction

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os

/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
CONFIG_PATH = '/root/work/artwork_sequence/train_test_configuration'

In [3]:
museum_sequence_path = {
    'x_train' : os.path.join(CONFIG_PATH, 'config_0/X_train.csv'),
    'x_test' : os.path.join(CONFIG_PATH, 'config_0/X_test.csv'),
    'x_train_matrix' : os.path.join(CONFIG_PATH, 'config_0/X_train_matrix.npy'),
    'x_test_matrix' : os.path.join(CONFIG_PATH, 'config_0/X_test_matrix.npy'),
    'weights_folder' : os.path.join(CONFIG_PATH, 'config_0/trained_model_weights')
}
museum_sequence_path

{'weights_folder': '/root/work/artwork_sequence/train_test_configuration/config_0/trained_model_weights',
 'x_test': '/root/work/artwork_sequence/train_test_configuration/config_0/X_test.csv',
 'x_test_matrix': '/root/work/artwork_sequence/train_test_configuration/config_0/X_test_matrix.npy',
 'x_train': '/root/work/artwork_sequence/train_test_configuration/config_0/X_train.csv',
 'x_train_matrix': '/root/work/artwork_sequence/train_test_configuration/config_0/X_train_matrix.npy'}

## Load data

In [4]:
df_x_train = pd.read_csv(museum_sequence_path['x_train'], index_col=0)
df_x_test = pd.read_csv(museum_sequence_path['x_test'], index_col=0)
x_train_matrix = np.load(museum_sequence_path['x_train_matrix'])
x_test_matrix = np.load(museum_sequence_path['x_test_matrix'])
df_x_train.head()

,tour_path
20,/root/work/datasets/artwork_sequence/rijksmuse...
7,/root/work/datasets/artwork_sequence/rijksmuse...
40,/root/work/datasets/artwork_sequence/prado_cra...
0,/root/work/datasets/artwork_sequence/rijksmuse...
23,/root/work/datasets/artwork_sequence/prado_cra...


In [5]:
x_train_matrix.shape

(637, 300)

## Reset Tensorflow session

In [6]:
tf.keras.backend.clear_session()

## Config data to fit with the model input

Because the **Prediction feature model** split the data into training and validation dataset, it is necessary to give all the data in only one block

**Define timeline**

In [7]:
time = np.arange(x_train_matrix.shape[0] + x_test_matrix.shape[0])
time.shape

(859,)

**Define configuration to deal with the windowed dataset**

In [8]:
split_time = x_train_matrix.shape[0]

X = np.concatenate((x_train_matrix, x_test_matrix))

#the length mean average of the tours
window_size = 5

batch_size = 128
shuffle_buffer_size = 300

## Create models

**Helper function to save model's weights**

In [9]:
def save_weights(model, index, museum_sequence_path):
    #Create the folder where the weights are saved
    model_feature_folder = os.path.join(museum_sequence_path['weights_folder'], 'model_feature_'+str(index))
    if not os.path.exists(model_feature_folder):
        os.makedirs(model_feature_folder)
    
    #Save weights
    model.save_weights(os.path.join(model_feature_folder, 'weights_feature_'+str(index)))

**Define model**

In [10]:
from Prediction_model_feature import Prediction_model_feature

In [11]:
n_features = X.shape[1]
models = []

In [12]:
from IPython.display import clear_output
import time

start_time = time.time()
for i in range(n_features):
    clear_output(wait=True)
    print("---------- Feature %s -------------" % (i))
    model_prediction = Prediction_model_feature(
        X=X[:, i],
        split_time=split_time,
        train_batch_size=batch_size, 
        val_batch_size=batch_size, 
        window_size=window_size, 
        shuffle_buffer=shuffle_buffer_size,
        name="feature " + str(i))
    model_prediction.define_model()
    model_prediction.train_model(epochs=40, lr=1e-6)
    models.append(model_prediction)
    
    #Save weights
    save_weights(model_prediction.get_model(), i, museum_sequence_path)
    
print("--- %s seconds ---" % (time.time() - start_time))

---------- Feature 0 -------------
Epoch 1/40
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
5/5 [==============================] - 5s 1s/step - loss: 5.9086 - mae: 6.3998 - val_loss: 0.0000e+00 - val_mae: 0.0000e+00
Epoch 2/40
5/5 [==============================] - 0s 42ms/step - loss: 2.5015 - mae: 2.9665 - val_loss: 1.6763 - val_mae: 2.1579
Epoch 3/40
5/5 [==============================] - 0s 42ms/step - loss: 1.3195 - mae: 1.7671 - val_loss: 0.6628 - val_mae: 1.0846
Epoch 4/40
5/5 [==============================] - 0s 42ms/step - loss: 0.8607 - mae: 1.2635 - val_loss: 0.6282 - val_mae: 1.0313
Epoch 5/40
5/5 [==============================] - 0s 41ms/step - loss: 0.6645 - mae: 1.0614 - val_loss: 0.6711 - val_mae: 1.0979
Epoch 6/40
5/5 [==============================] - 0s 41ms/step - loss: 0.5959 - mae: 1.0001 - val_loss: 0.4876 - val_mae: 0.8912
Epoch 7/40
5/5 [==============================] - 0s 40ms/step - loss: 0.5195 - mae: 0.9064

5/5 [==============================] - 0s 39ms/step - loss: 0.3462 - mae: 0.7301 - val_loss: 0.3362 - val_mae: 0.7268
Epoch 38/40
5/5 [==============================] - 0s 40ms/step - loss: 0.3459 - mae: 0.7298 - val_loss: 0.3360 - val_mae: 0.7267
Epoch 39/40
5/5 [==============================] - 0s 39ms/step - loss: 0.3455 - mae: 0.7294 - val_loss: 0.3359 - val_mae: 0.7265
Epoch 40/40
5/5 [==============================] - 0s 38ms/step - loss: 0.3452 - mae: 0.7291 - val_loss: 0.3358 - val_mae: 0.7263
--- 15.438527345657349 seconds ---


In [ ]:
len(models)

In [ ]:
models[0].model.summary()

## Evaluate model

### Load decoder model

In [ ]:
from tensorflow.python.keras.models import load_model

In [ ]:
decoder_model = load_model('wasserstein_decoder.h5')
decoder_model.summary()

**Test decoder with example**

In [ ]:
p = decoder_model.predict(X[1].reshape((1,1,1,-1)))

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(p[0][...,::-1])

### Predict features

**Sliced validation dataset**

In [ ]:
def validation_dataset(series, window_size, batch_size):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.map(lambda w: (w[:-1], w[-1:]))
    return ds.batch(batch_size).prefetch(1)

**Predict features**

In [ ]:
prediction = []

for i in range(n_features):
    val_dataset = validation_dataset(x_valid[:,i], window_size, batch_size)
    for x, y in val_dataset.take(1):
        prediction_feature = models[i].model.predict(x)[0]
        prediction.append(prediction_feature)

In [ ]:
code = []
for p in prediction:
    code.append(prediction[0][0])

code = np.array(code)

In [ ]:
code = code.reshape((-1,))

**Decode code**

In [ ]:
p = decoder_model.predict(code.reshape((1,1,1,-1)))

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(p[0])

In [ ]:
valid_decode = decoder_model.predict(x_valid[0].reshape((1,1,1,-1)))

In [ ]:
plt.imshow(valid_decode[0][...,::-1])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

cosine_distances(code, x_valid[0][...,::-1])

### Plot feature prediction

In [ ]:
from utils_plot import plot_series, plot_train_history, plot_prediction
import tensorflow as tf

In [ ]:
val_dataset = validation_dataset(x_valid[:,0], window_size, batch_size)
for x, y in val_dataset.take(1):
    prediction = models[0].model.predict(x)[0]
    plot = plot_prediction([x[0].numpy(), y[0].numpy(), prediction[0]] , 'Simple LSTM model')
    plot.show()

In [ ]:
val_dataset = validation_dataset(x_valid[:,20], window_size, batch_size)
for x, y in val_dataset.take(1):
    prediction = models[20].model.predict(x)[0]
    plot = plot_prediction([x[0].numpy(), y[0].numpy(), prediction[0]] , 'Simple LSTM model')
    plot.show()

**Predict x_valid**

In [ ]:
def model_forecast(model, series, window_size, batch_size):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size))
    ds = ds.batch(batch_size).prefetch(1)
    forecast = model.predict(ds)
    return forecast

In [ ]:
x = np.concatenate((x_train[:, 1], x_valid[:, 1]))

In [ ]:
rnn_forecast = model_forecast(model_prediction.model, x, window_size, batch_size)

In [ ]:
rnn_forecast = rnn_forecast[split_time-window_size+1:,-1,0]

In [ ]:
plot_series(time_valid, [rnn_forecast], label="rnn")